In [ ]:
import json
import numpy as np
import random
import pypuf.io
import pypuf.simulation
import time
import hashlib
import busio
import board
import digitalio
import pickle
import adafruit_rfm9x
import zlib
import time
from digitalio import DigitalInOut, Direction, Pull
import adafruit_ssd1306
import subprocess
import os
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
from tflite_runtime import interpreter
import librosa

# Initialize the OLED display
i2c = board.I2C()
oled = adafruit_ssd1306.SSD1306_I2C(128, 32, i2c)

# Function to update the OLED display with status
def update_display(status):
    oled.fill(0)  # Clear the display
    oled.text("Car Status:",10, 5, 1)
    oled.text(status,10, 15, 1)
    oled.show()  # Update the OLED display


RADIO_FREQ_MHZ = 433.0
CS = digitalio.DigitalInOut(board.CE1)
RESET = digitalio.DigitalInOut(board.D25)

spi = busio.SPI(board.SCK, MOSI=board.MOSI, MISO=board.MISO)
rfm9x = adafruit_rfm9x.RFM9x(spi, CS, RESET, RADIO_FREQ_MHZ)

rfm9x.tx_power = 23
# enable CRC checking
rfm9x.enable_crc = True
# set delay before transmitting ACK (seconds)
rfm9x.ack_delay = 0.1
# set node addresses
rfm9x.node = 1
rfm9x.destination = 2
rfm9x.ack_retries = 5
rfm9x.ack_wait = 2.0  # 2 seconds

puf_BITS = 32
puf_SEED = 1
puf_SETS = 8

# Load your TFLite model
model = interpreter.Interpreter(model_path="/home/hello/Desktop/PUF_RKESystem-main/RF/CNN-90per.tflite")
model.allocate_tensors()

# Get input and output details
input_details = model.get_input_details()
output_details = model.get_output_details()

# Get the input and output tensor shapes and data types
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']
input_dtype = input_details[0]['dtype']
output_dtype = output_details[0]['dtype']

# Load and preprocess the WAV file
def preprocess_wav(file_path, input_shape):
    # Load the audio file using librosa
    audio, sample_rate = librosa.load(file_path, sr=None)
    
    # Ensure the audio length matches the expected input shape
    if len(audio) != input_shape[1]:
        if len(audio) < input_shape[1]:
            # If the audio is shorter than expected, pad it with zeros on both sides to match the input shape
            padding = input_shape[1] - len(audio)
            audio = np.pad(audio, (0, padding), 'constant')
        else:
            # If the audio is longer than expected, trim it down to match the input shape
            audio = audio[:input_shape[1]]
    return audio

def trim_and_pad_audio(file_path, output_path, desired_duration=300, min_silence_len=100, silence_thresh=-30):
    """
    Trims silence from the start and end of an audio file and ensures the resulting file is exactly the desired duration (in ms).
    If the audio is shorter than the desired duration, it is padded with silence evenly at both the beginning and end.
    
    Parameters:
    - file_path: Path to the WAV file to process.
    - output_path: Path where the trimmed and padded audio will be saved.
    - desired_duration: Desired duration of the resulting audio file after trimming and padding (in ms).
    - min_silence_len: Minimum length of silence to be used for detection (in ms).
    - silence_thresh: Silence threshold (in dBFS).
    """
    audio = AudioSegment.from_wav(file_path)
    
    # Detect nonsilent parts
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    
    # If no nonsilent ranges are detected, the audio is probably all silence.
    if not nonsilent_ranges:
        raise ValueError("No non-silent segments detected in the audio.")
    
    # Only consider the start of the first non-silent segment and the end of the last non-silent segment.
    start, _ = nonsilent_ranges[0]
    _, end = nonsilent_ranges[-1]
    signal = audio[start:end]

    # Calculate the actual duration of the trimmed audio
    actual_duration = len(signal)
    if actual_duration > desired_duration:
        # Trim the audio to the desired duration
        start_trim = (actual_duration - desired_duration) // 2
        end_trim = (actual_duration - desired_duration) - start_trim
        signal = signal[start_trim:-end_trim]
    elif actual_duration < desired_duration:  
        # Calculate the required padding on both sides
        padding_duration = desired_duration - actual_duration
        left_padding_duration = padding_duration // 2
        right_padding_duration = padding_duration - left_padding_duration
        
        # Create silence for padding
        left_padding_silence = AudioSegment.silent(duration=left_padding_duration)
        right_padding_silence = AudioSegment.silent(duration=right_padding_duration)
        
        # Add padding to both the beginning and end
        signal = left_padding_silence + signal + right_padding_silence
    else:
        # The audio is already the desired duration
        pass
    # Save the trimmed and padded audio
    signal.export(output_path, format="wav")


def send_message(key, value):
    print(f"Sending {key} from Car to Key...")
    if not isinstance(value, bytes):
        value = bytes(value, 'utf-8')
    
    MAX_CHUNK_SIZE = 170  # Adjusted chunk size
    num_chunks = (len(value) + MAX_CHUNK_SIZE - 1) // MAX_CHUNK_SIZE
    for i in range(num_chunks):
        chunk = value[i * MAX_CHUNK_SIZE: (i + 1) * MAX_CHUNK_SIZE]
        rfm9x.send_with_ack(chunk)  # Send data as bytes
        if i < num_chunks - 1:  # If there are more chunks to send, add a delay
            time.sleep(0.1)  # Adjust this delay as needed
    print(f"{key} has been sent")


# Replace receive_message with LoRa reception
def receive_message(key):
    print(f"Receiving {key} from Key to Car...")
    while True:
        data = rfm9x.receive(with_ack=True)
        if data is not None:
            print(f"Received {key}")
            return data
        time.sleep(1)  # Add a small delay before checking again

def hackrf_capture_start():
    """
    Starts capturing signals using HackRF.
    Returns the started process.
    """
    hackrf_process = subprocess.Popen([
        "hackrf_transfer",
        "-w",  # Save as WAV file
        "-f", "433000000",  # Frequency: 433 MHz
        "-s", "2000000",    # Sample rate: 2.0 MSPS
        "-l", "16",         # IF gain: 16
        "-g", "14"          # Baseband gain: 14
    ])
    return hackrf_process

def hackrf_capture_stop(hackrf_process):
    """
    Stops the HackRF capture process.
    """
    hackrf_process.terminate()
    hackrf_process.wait()
    print("HackRF capture complete.")
    
def remove_files_in_folder(folder):
    """Remove all WAV files in the specified folder."""
    for filename in os.listdir(folder):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder, filename)
            os.remove(file_path)

def move_and_trim_files(src_folder, dest_folder):
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Remove all WAV files in the destination folder
    remove_files_in_folder(dest_folder)

    # Iterate through all files in the source directory
    for filename in os.listdir(src_folder):
        if filename.endswith(".wav"):
            src_path = os.path.join(src_folder, filename)
            dest_path = os.path.join(dest_folder, filename)
            
            # Load the audio file to check its duration
            audio = AudioSegment.from_wav(src_path)
            
            # Check if the audio duration is longer than 5 seconds
            if len(audio) > 5000:  # Duration is in milliseconds
                # Trim the audio to the last 5 seconds
                start_time = len(audio) - 5000  # Start time for the last 5 seconds
                trimmed_audio = audio[start_time:]
                
                # Export the trimmed audio
                trimmed_audio.export(dest_path, format="wav")
                
                # Now proceed to trim_and_pad_audio for further processing
                trim_and_pad_audio(dest_path, dest_path)
            else:
                # Audio is shorter than 5 seconds, use trim_and_pad_audio directly
                trim_and_pad_audio(src_path, dest_path)
            
            # Remove the original file from the source folder
            os.remove(src_path)
            
            
while True:
    with open('car_data.pkl', 'rb') as f:
        data = pickle.load(f)
    remove_files_in_folder(os.getcwd())
    hackrf_process = hackrf_capture_start()

    key_id = data['Key_id']
    challenge_ndarray = data['Challenge']

    response_str = data['Response']
    Ks = data['Ks']

    # Process Ks to get Ks_ndarray
    Ks_ndarray = np.array([int(c) for c in Ks], dtype=np.int8)

    """
    Receive MA1
    """

    received_data = receive_message('MA1_data').decode('utf-8')
    start_time = time.time()
    message_data = json.loads(received_data)

    # Stop the HackRF capture
    hackrf_capture_stop(hackrf_process)
    current_folder = os.getcwd()  # Gets the current directory
    destination_folder = "signal"  # Replace with your desired path
    move_and_trim_files(current_folder, destination_folder)
    signal_folder = "signal/"  # Specify the folder where the WAV file is located

    # List all files in the signal folder
    signal_files = os.listdir(signal_folder)

    # Filter for WAV files
    wav_files = [f for f in signal_files if f.endswith(".wav")]

    if len(wav_files) == 1:
        # Only proceed if there is exactly one WAV file in the folder
        wav_file_path = os.path.join(signal_folder, wav_files[0])

        # Preprocess the WAV file
        input_data = preprocess_wav(wav_file_path, input_shape)
        # Reshape the audio data to match the expected input shape
        input_data =  np.expand_dims(np.expand_dims(input_data, axis=0), axis=-1)
        # Set the input tensor
        model.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        model.invoke()

        # Get the output tensor
        output_data = model.get_tensor(output_details[0]['index'])

        # # Assuming a threshold of 0.5 for binary classification
        threshold = 0.60
        binary_prediction = (output_data > threshold).astype(np.int32)

        # # Output confidence (probability)
        confidence = output_data[0][0]

        # # Use binary_prediction as your model's prediction and confidence as the probability
        print("Predicted class:", binary_prediction)
        print("Confidence:", confidence)
        # print(output_data)
        # predict_result=output_data.argmax()
       
        # Continue with the rest of code 2 depending on binary_prediction
        if binary_prediction == 1:
            # Code to execute when binary_prediction is 1
            print("Proceed with code 2")
            # Insert the remaining part of code 2 here
        else:
            remove_files_in_folder(os.getcwd())
            # Code to execute when binary_prediction is not 1
            print("Predicted class is not 1. Returning to start of while loop.")
            continue  # Return to the start of the while loop to wait for "MA1_data"
    else:
        print("There are no/above 1 WAV files in the 'signal' folder or there are multiple WAV files.")
        continue
    key_id = message_data['ID']
    Ni_encrypted = bytes.fromhex(message_data['Encrypted_Ni'])  # Convert hex string back to bytes

    """
    Decode Ni
    """
    Ni_encrypted = np.frombuffer(Ni_encrypted, dtype=np.int8).reshape(puf_BITS) # byte-> numpy array

    Ni = Ni_encrypted ^ Ks_ndarray # decode(XOR)

    Ni = np.array2string(Ni, separator='', prefix='', suffix='')[1:-1] # remove brackets and whitespace, type: numpy array-> string


    """
    Generate & XOR Nc
    """
    Nc = bin(random.getrandbits(puf_BITS))[2:].zfill(puf_BITS) #type: string

    Nc_ndarray = np.array([int(c) for c in Nc], dtype= np.int8) #type: string -> numpy array

    Nc_encrypted = Nc_ndarray ^ Ks_ndarray #type: numpy array
    Nc_encryptedz_compressed = zlib.compress(Nc_encrypted.tobytes())
    """
    XOR challenge
    """


    challenge_encrypted = challenge_ndarray ^ Ks_ndarray #type: numpy array

    """
    Generate hash
    """
    A0 = key_id + Ni + Ks + Nc

    A0_hash_object = hashlib.sha256(A0.encode('utf-8'))

    A0_hex_dig = A0_hash_object.hexdigest()
    """
    Generate & XOR challenge new
    """
    challenge_new = pypuf.io.random_inputs(puf_BITS, puf_SETS, puf_SEED) # Generate Challenge, type: numpy array

    challenge_new_encrypted = challenge_new ^ Ks_ndarray #type: numpy array
    challenge_encrypted_compressed = zlib.compress(challenge_encrypted.tobytes())
    challenge_new_encrypted_compressed = zlib.compress(challenge_new_encrypted.tobytes())

    """
    send MA2
    """
    # Group the data into a dictionary
    message_data = {
        "A0": A0_hex_dig,
        "Encrypted_Nc": Nc_encryptedz_compressed.hex(),
        "Encrypted_Challenge": challenge_encrypted_compressed.hex(),
        "Encrypted_Challenge_New": challenge_new_encrypted_compressed.hex()
    }

    # Convert the dictionary to a JSON string and send
    message_json = json.dumps(message_data).encode('utf-8')

    send_message("Combined_Data", zlib.compress(message_json))

    """
    receive MA3
    """


    compressed_data = receive_message("Compressed_MA3_Data")
    decompressed_data = zlib.decompress(compressed_data)

    # Parse the combined data from JSON
    combined_data = json.loads(decompressed_data.decode('utf-8'))

    # Extract individual components
    A1_hex_dig = combined_data["A1"]
    response_encrypted = bytes.fromhex(combined_data["Encrypted_Response"])
    response_new_encrypted = bytes.fromhex(combined_data["Encrypted_Response_New"])
    cmd = combined_data["Command"]
    expiration_time = combined_data["ExpirationTime"]
    
    now_time = int(time.time())
    
    # Update the car status based on the command
    if cmd == "lock":
        car_status = "Locked"
        # Update the OLED display with the status
        update_display(car_status)
    elif cmd == "unlock":
        car_status = "Unlocked"
        # Update the OLED display with the status
        update_display(car_status)
    else:
        # Handle other commands as needed
        car_status = "Unknown"  # Set a default status

    
    if expiration_time >= now_time:

        """
        Decode Response & Response new
        """

        response_encrypted = np.frombuffer(response_encrypted, dtype=np.int8) # byte-> numpy array

        response = response_encrypted ^ Ks_ndarray[0:8] # decode(XOR)

        response_new_encrypted = np.frombuffer(response_new_encrypted, dtype=np.int8) # byte-> numpy array

        response_new = response_new_encrypted ^ Ks_ndarray[0:8] # decode(XOR)

        """
        Verify Response and A1
        """

        puf = pypuf.simulation.XORArbiterPUF(puf_BITS, puf_SEED)

        response_verified = puf.eval(challenge_ndarray) # produce response, type: numpy array
        #Above should compare with response


        response_str = np.array2string(response, separator='', prefix='', suffix='')[1:-1] #type: numpy array-> string

        response_new_str = np.array2string(response_new, separator='', prefix='', suffix='')[1:-1] #type: numpy array-> string

        A1_verified = key_id + Nc + Ks + response_str + response_new_str

        A1_verified_hash_object = hashlib.sha256(A1_verified.encode('utf-8'))

        A1_verified_hex_dig = A1_verified_hash_object.hexdigest()
        if(A1_verified_hex_dig == A1_hex_dig):
            if(np.array_equal(response, response_verified)):
                # Print or use the car_status variable as needed in your code.
                print("Car Status:", car_status)
                data = {
                        'Key_id': key_id,
                        'Challenge': challenge_new,
                        'Response': response_new,
                        'Ks': Ks
                }
                # print(data)
                # Serialize the register_output dictionary and write to car_data.pkl
                with open('car_data.pkl', 'wb') as f:
                    pickle.dump(data, f)
                        
            else:
                print("Failed Response verified")
        else:
            print("Failed A1 verified")
        
        
        print(time.time()- start_time )
        time.sleep(3)
    else:
        print("time was outdated")
    
    time.sleep(1)
